
# Creating stimuli for an auditory oddball experiment

Here we will follow the methods section from [Schlossmacher et al. (2022)](https://doi.org/10.1016/j.neuroimage.2022.119445)
to create a classic oddball (deviance detection) experiment. We will go over the methods section sentence by sentence,
and will recreate the experiment using *thebeat*.

Here is the section in full:

> Stimuli consisted of pure sine-tones of 600, 800, 1000, 1200, and 1400 Hz. Stimulus duration amounted to 100 ms, including rise/fall times of 10 ms. The sound volume was chosen to be easily audible during functional sequences but not unpleasantly loud. In oddball blocks, the 800 and 1200 Hz tones served as deviant and standard counterbalanced across participants. The probability of deviant to standard stimuli was 20:80 and stimulus presentation was pseudorandomized so that no two deviants were presented consecutively. In control blocks, all five tone stimuli were presented randomly with a probability of 20% (see Fig. 1A). Throughout, participants’ task was to respond to target stimuli of 300 ms duration, which were randomly interspersed in the sequence. In total, 40 targets were included, the frequency of which conformed to the stimulus probability of the current block, i.e., all tone frequencies could be potential targets. We included this task in order to ensure that participants attended to the tone sequence but did not view the deviant as a target. Interstimulus intervals (ISIs) ranged from 1.04 to 17.25 s (M = 3.01, SD = 2.06) and were derived using optseq2 (Greve, 2009). Four different optseq sequences were computed and randomly assigned to two runs per participant, creating 12 different sequence combinations. In total, two runs of 250 stimuli (ca. 13 min each) were acquired and separated by a short break. One run could start with either an oddball or control block, which seamlessly changed to control or oddball block in the middle of the run. Whether the participants started with the oddball or control block was counterbalanced across participants. The second run was always presented in reverse order, e.g., if the first run was oddball-control, the second run was control-oddball. Before starting the experiment, a short practice block of about 1 min was presented in order to accustom the participants to the task. Stimuli in this practice block were structured like the experimental block that followed, i.e., they also included brief oddball and control sequences. At all times, a white fixation cross was presented on a black screen, and participants were asked to fixate during the run. Stimulus presentation and response collection were controlled by the software Presentation (version 21.1, Neurobehavioral Systems, Albany, CA).

In [2]:
from thebeat.core import Sequence, Stimulus
import numpy as np

rng = np.random.default_rng(seed=123)

ModuleNotFoundError: No module named 'thebeat.resources'

In [ ]:
stim_600 = Stimulus.generate()